# Sulgudega lausete kogumine korpusest

Teeme koondkorpus_base_v2_20220216.tsv failist faili, kus on sees laused mis sisaldavad sulge, nende collection_id ja esimese sõna span.start.

Laused tuleb koguda originaalkujul

In [1]:
from pathlib import Path

Path("./result").mkdir(parents=True, exist_ok=True) 

collectionName = 'koondkorpus_base_v2_20220216' # 'koondkorpus_base_v2'
corpusTSV = f'/Users/rabauti/repos/estnltk-model-training/collocation_net/data_extraction/from_tsv/koondkorpus_base_v2_20220216.tsv'
outputTSV = f'result/1_sentences_with_brackets.tsv'

fOut = open(outputTSV, 'w')

In [2]:

def getSentenceText(current_sentence):
    current_sentence_words = []
    for i in range(len(current_sentence)):
        w = ''
        if i==0:
            w = current_sentence[i]['text']
        else:
            spaces = int(current_sentence[i]['start']) - int(current_sentence[i-1]['start'])- len(current_sentence[i-1]['text'])
            for j in range(spaces):
                w = w + ' '
            w = w + current_sentence[i]['text']
        current_sentence_words.append(w) 
    return ''.join(current_sentence_words) 
    

#text sisaldab 
import re
def hasBrackets(sentence_text):
    #if re.search(r'(?<=abc)def', sentence_text):
    if re.search(r' \( [^\(\)]+ \)', ' ' + sentence_text):
        return True
    else:
        return False

    

In [3]:
%%time

has_brackets = False

count = 0
current_sentence = []

colId = None
prevCol = 0

with open(corpusTSV) as f:
    for line in f:
        count +=1
        line = line.strip("\n")
        row = line.split('\t')
        
        if not len(row) == 9:
            continue
        
        data = {}
        prevCol = colId
        (colId, data['start'], data['id'], data['text'], data['lemma'], data['upostag'], data['deprel'], data['head'], data['feat']) = row
        colId = int(colId)
        
        for k in ('id', 'start', 'head'):
            data[k] = int(data[k])
        
        
        #sentence start
        if data['id'] == 1:
            if len(current_sentence):
                current_sentence_text = getSentenceText(current_sentence) 
                has_brackets = hasBrackets(current_sentence_text)
                if has_brackets:
                    fOut.write('%s\t%s\t%s\n' % (prevCol, current_sentence[0]['start'], current_sentence_text,  ))
            current_sentence = []
            has_brackets = False
          
        current_sentence.append(data)

    
    #viimane lause
    current_sentence_text = getSentenceText(current_sentence) 
    if  hasBrackets(current_sentence_text):           
        fOut.write('%s\t%s\t%s\n' % (prevCol, current_sentence[0]['start'], current_sentence_text,  ))

fOut.close()
f.close()


CPU times: user 15min 38s, sys: 8.3 s, total: 15min 47s
Wall time: 15min 52s
